In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [ ]:
# 공개 데이터셋에서 학습 데이터를 내려받습니다.
train_data = datasets.FashionMNIST(
    root="/content/drive/MyDrive/Study/00_pytorch/data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.
test_data = datasets.FashionMNIST(
    root="/content/drive/MyDrive/Study/00_pytorch/data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [ ]:
batch_size = 12

train_dataloader = DataLoader(
    train_data,
    batch_size=batch_size,
    num_workers=15,
    pin_memory=False)
test_dataloader = DataLoader(
    test_data,
    batch_size=batch_size,
    num_workers=15,
    pin_memory=False)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W] : ", X.shape)
    print("Shape of y : ", y.shape, y.dtype)
    break

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 15 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Shape of X [N, C, H, W] :  torch.Size([12, 1, 28, 28])
Shape of y :  torch.Size([12]) torch.int64


In [ ]:
# 학습에 사용할 CPU나 GPU 장치를 얻습니다.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# 모델을 정의합니다.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model):
    size = len(dataloader.dataset)
    # model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 15 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Streaming output truncated to the last 5000 lines.
loss: 0.508355  [ 4800/60000]
loss: 0.756824  [ 6000/60000]
loss: 0.804234  [ 7200/60000]
loss: 0.864321  [ 8400/60000]
loss: 0.967574  [ 9600/60000]
loss: 0.709969  [10800/60000]
loss: 0.777143  [12000/60000]
loss: 0.858473  [13200/60000]
loss: 0.502104  [14400/60000]
loss: 0.483840  [15600/60000]
loss: 1.750969  [16800/60000]
loss: 0.752943  [18000/60000]
loss: 0.273100  [19200/60000]
loss: 0.827561  [20400/60000]
loss: 0.719850  [21600/60000]
loss: 0.632548  [22800/60000]
loss: 0.359993  [24000/60000]
loss: 0.448296  [25200/60000]
loss: 0.098610  [26400/60000]
loss: 1.356618  [27600/60000]
loss: 0.480154  [28800/60000]
loss: 0.318527  [30000/60000]
loss: 1.284246  [31200/60000]
loss: 0.434679  [32400/60000]
loss: 0.451324  [33600/60000]
loss: 0.535649  [34800/60000]
loss: 0.490459  [36000/60000]
loss: 0.465442  [37200/60000]
loss: 0.592781  [38400/60000]
loss: 0.319457  [39600/60000]
loss: 0.533759  [40800/60000]
loss: 0.607257  [42

In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/Study/00_pytorch/model_save/model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [ ]:
model_loaded = torch.load("/content/drive/MyDrive/Study/00_pytorch/model_save/model.pth")
model_loaded.keys()

odict_keys(['linear_relu_stack.0.weight', 'linear_relu_stack.0.bias', 'linear_relu_stack.2.weight', 'linear_relu_stack.2.bias', 'linear_relu_stack.4.weight', 'linear_relu_stack.4.bias'])